In [1]:
from nep_loss import multiclass_logloss
from sklearn.metrics import accuracy_score

# Data Encoding and Tokenizing

In [2]:
import pickle

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [4]:
with open('../data.nosync/ebert_test_train_data.pkl', 'rb') as f:
    train_data, test_data = pickle.load(f)

In [5]:
train_data, valid_data = train_test_split(train_data, stratify=train_data['stars'], test_size=0.1, random_state=42)

In [6]:
X_train = train_data['review']
y_train = train_data['stars']
X_valid = valid_data['review']
y_valid = valid_data['stars']
X_test = test_data['review']
y_test = test_data['stars']

len(X_train), len(y_train), len(X_valid), len(y_valid), len(X_test), len(y_test)

(5204, 5204, 579, 579, 1928, 1928)

In [7]:
lbl_enc = preprocessing.LabelEncoder()
y_encoded = lbl_enc.fit_transform(list(y_train) + list(y_valid) + list(y_test))
y_train_enc = y_encoded[0:len(y_train)]
y_valid_enc = y_encoded[len(y_train):len(y_train) + len(y_valid)]
y_test_enc = y_encoded[len(y_train) + len(y_valid):len(y_train) + len(y_valid) + len(y_test)]

# TF-IDF

https://www.kaggle.com/code/abhishek/approaching-almost-any-nlp-problem-on-kaggle/notebook

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(X_train) + list(X_valid))
xtrain_tfv =  tfv.transform(X_train) 
xvalid_tfv = tfv.transform(X_valid)

/Users/briankim/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/_param_validation.py:594: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(

KeyboardInterrupt



## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0, max_iter=1000)
clf.fit(xtrain_tfv, y_train_enc)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
print("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

In [ ]:
with open('scores.txt', 'w') as f:
    f.write("Logistic Regression logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))

## Word Counts

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(X_train) + list(X_valid))
xtrain_ctv =  ctv.transform(X_train) 
xvalid_ctv = ctv.transform(X_valid)

In [ ]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, y_train_enc)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
print("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

In [ ]:
with open('scores.txt', 'a') as f:
    f.write("Logistic Regression + Word Counts logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
    f.write("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, y_train_enc)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
print("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

In [ ]:
with open('scores.txt', 'a') as f:
    f.write("Naive Bayes + TF-IDF logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
    f.write("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

## Naive Bayes + Word Count

In [ ]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, y_train_enc)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
print("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

In [ ]:
with open('scores.txt', 'a') as f:
    f.write("Naive Bayes + Word Counts logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
    f.write("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

## SVM

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC

In [ ]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [ ]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, y_train_enc)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
print("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

In [ ]:
with open('scores.txt', 'a') as f:
    f.write("SVM + TF-IDF logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
    f.write("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

## XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), y_train_enc)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
print("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

In [ ]:
with open('scores.txt', 'a') as f:
    f.write("XGBoost + TF-IDF logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
    f.write("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

## XGBoost + SVD

In [ ]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, y_train_enc)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
print("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

In [ ]:
with open('scores.txt', 'a') as f:
    f.write("XGBoost + SVD logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
    f.write("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

# Grid Search

## SVD + Scaling + Logistic Regression

In [ ]:
from sklearn import preprocessing, metrics, pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

In [ ]:
svd = TruncatedSVD()
scl = preprocessing.StandardScaler()
lr_model = LogisticRegression()
clf = pipeline.Pipeline(
    [
        ('svd', svd),
        ('scl', scl),
        ('lr', lr_model)
    ]
)

In [ ]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2'],
              'lr__max_iter': [100, 1000],
             }

In [ ]:
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1,
#                      iid=True,
                     refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, y_train_enc)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

## XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
# clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
#                         subsample=0.8, nthread=10, learning_rate=0.1)

xbg_model = xgb.XGBClassifier(nthread=-1)

# Create the pipeline 
clf = pipeline.Pipeline([('xgb', xgb_model)])

# parameter grid
param_grid = {
    'xgb__max_depth': list(range(10)),
    'xgb__n_estimators': [10, 50, 100, 200, 500],
    'xgb__colsample_bytree': [0.1, 0.3, 0.5, 0.8, 0.9],
    'xgb__subsample': [0.1, 0.3, 0.5, 0.8, 0.9],
    'xgb__learning_rate': [0.001, 0.01, 0.1, 0.5],
}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1,
#                      iid=True,
                     refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, y_train_enc)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1,
#                      iid=True,
                     refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, y_train_enc)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Word Vectors

## Glove Vectors

In [13]:
from tqdm import tqdm
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [14]:
# from http://www-nlp.stanford.edu/data/glove.840B.300d.zip

embeddings_index = {}
with open('../../glove.840B.300d.txt') as f:
    i = 0
    for line in tqdm(f):
        try:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
        except Exception as e:
            print(i)
            print(values[:10])
        i += 1
        
print("Found %s words vectors" % len(embeddings_index))

54872it [00:02, 19726.84it/s]

52343
['.', '.', '.', '-0.1573', '-0.29517', '0.30453', '-0.54773', '0.098293', '-0.1776', '0.21662']


130611it [00:06, 18150.10it/s]

128261
['at', 'name@domain.com', '0.0061218', '0.39595', '-0.22079', '0.78149', '0.38759', '0.28888', '0.18495', '-0.37328']


153588it [00:08, 18882.07it/s]

...

2058966
['at', 'Amazon.com', '-0.5275', '-0.73685', '0.10968', '0.22214', '-0.30063', '-0.63201', '-0.053204', '-0.16241']


2168904it [01:59, 19252.00it/s]

2165246
['is', 'name@domain.com', '-0.1197', '0.10706', '-0.10519', '-0.12412', '0.4096', '-0.0287', '0.34704', '0.3549']


2196017it [02:00, 18235.89it/s]

Found 2195884 words vectors


In [17]:
def sentence2vector(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if w not in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    else:
        return v / np.sqrt((v ** 2).sum())

In [18]:
xtrain_glove = [sentence2vector(x) for x in X_train]
xvalid_glove = [sentence2vector(x) for x in X_valid]

In [20]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [23]:
import xgboost as xgb

In [26]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, y_train_enc)
predictions = clf.predict_proba(xvalid_glove)

print("logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
print("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

/Users/briankim/opt/anaconda3/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [21:58:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


logloss: 1.881 
accuracy: 0.364


In [28]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, y_train_enc)
predictions = clf.predict_proba(xvalid_glove)

print("logloss: %0.3f " % multiclass_logloss(y_valid_enc, predictions))
print("accuracy: %0.3f" % accuracy_score(y_valid_enc, np.argmax(predictions, axis=1)))

/Users/briankim/opt/anaconda3/lib/python3.8/site-packages/xgboost/core.py:160: UserWarning: [21:58:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


logloss: 1.749 
accuracy: 0.387
